In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/var/folders/zk/jxl4jn3s41b8q_1hhx9_8k940000gn/T/ipykernel_3202/3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [23]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk


- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [28]:
## Read Data for the Fraudulent Email Kaggle Challenge
# Load the train and test datasets
train_data = pd.read_csv('data/kg_train.csv')
test_data = pd.read_csv('data/kg_test.csv')
# Reduce the training set to speed up development. 
# Modify for final system
train_data = train_data.head(1000)
print(train_data.shape)
train_data.fillna("",inplace=True)

(1000, 2)


In [29]:
test_data = test_data.head(1000)
print(test_data.shape)
test_data.fillna("",inplace=True)

(1000, 1)


In [30]:
train_data.columns

Index(['text', 'label'], dtype='object')

In [31]:
test_data.columns

Index(['text'], dtype='object')

### Let's divide the training and test set into two partitions

In [32]:
# Separate features and labels from training data
X_train = train_data['text']
y_train = train_data['label']

# Test data only has the text column
X_test = test_data['text']

## Data Preprocessing

In [33]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [ ]:
# Function to clean HTML content
def clean_html(text):
    # Remove inline JavaScript/CSS
    text = re.sub(r'<script.*?>.*?</script>', '', text, flags=re.DOTALL)
    text = re.sub(r'<style.*?>.*?</style>', '', text, flags=re.DOTALL)
    
    # Remove HTML comments
    text = re.sub(r'<!--.*?-->', '', text, flags=re.DOTALL)
    
    # Remove remaining HTML tags
    text = re.sub(r'<[^>]+>', '', text)
    
    return text

- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [ ]:
# Function to clean HTML content and preprocess text
def clean_html_and_text(text):
    # Remove inline JavaScript/CSS
    text = re.sub(r'<script.*?>.*?</script>', '', text, flags=re.DOTALL)
    text = re.sub(r'<style.*?>.*?</style>', '', text, flags=re.DOTALL)
    
    # Remove HTML comments
    text = re.sub(r'<!--.*?-->', '', text, flags=re.DOTALL)
    
    # Remove remaining HTML tags
    text = re.sub(r'<[^>]+>', '', text)
    
    # Remove all special characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    
    # Remove all single characters
    text = re.sub(r'\b[a-zA-Z]\b', '', text)
    
    # Remove single characters from the start
    text = re.sub(r'^\s*[a-zA-Z]\s+', '', text)
    
    # Substitute multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)
    
    # Remove prefixed 'b'
    text = re.sub(r'^b\s+', '', text)
    
    # Convert to lowercase
    text = text.lower()
    
    return text

## Now let's work on removing stopwords
Remove the stopwords.

In [34]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Function to clean HTML content and preprocess text
def clean_html_and_text(text):
    # Remove inline JavaScript/CSS
    text = re.sub(r'<script.*?>.*?</script>', '', text, flags=re.DOTALL)
    text = re.sub(r'<style.*?>.*?</style>', '', text, flags=re.DOTALL)
    
    # Remove HTML comments
    text = re.sub(r'<!--.*?-->', '', text, flags=re.DOTALL)
    
    # Remove remaining HTML tags
    text = re.sub(r'<[^>]+>', '', text)
    
    # Remove all special characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    
    # Remove all single characters
    text = re.sub(r'\b[a-zA-Z]\b', '', text)
    
    # Remove single characters from the start
    text = re.sub(r'^\s*[a-zA-Z]\s+', '', text)
    
    # Substitute multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)
    
    # Remove prefixed 'b'
    text = re.sub(r'^b\s+', '', text)
    
    # Convert to lowercase
    text = text.lower()
    
    # Remove stopwords
    text = ' '.join([word for word in text.split() if word not in stop_words])
    
    return text

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/victorg/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [40]:
lemmatizer = WordNetLemmatizer()

def clean_html_and_text(text):
    # Remove inline JavaScript/CSS
    text = re.sub(r'<script.*?>.*?</script>', '', text, flags=re.DOTALL)
    text = re.sub(r'<style.*?>.*?</style>', '', text, flags=re.DOTALL)
    
    # Remove HTML comments
    text = re.sub(r'<!--.*?-->', '', text, flags=re.DOTALL)
    
    # Remove remaining HTML tags
    text = re.sub(r'<[^>]+>', '', text)
    
    # Remove all special characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    
    # Remove all single characters
    text = re.sub(r'\b[a-zA-Z]\b', '', text)
    
    # Remove single characters from the start
    text = re.sub(r'^\s*[a-zA-Z]\s+', '', text)
    
    # Substitute multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)
    
    # Remove prefixed 'b'
    text = re.sub(r'^b\s+', '', text)
    
    # Convert to lowercase
    text = text.lower()
    
    # Remove stopwords
    text = ' '.join([word for word in text.split() if word not in stop_words])
    
    # Lemmatize words
    text = ' '.join([lemmatizer.lemmatize(word) for word in text.split()])
    
    return text

In [48]:
# Clean the HTML content and preprocess the text columns
train_data['text'] = train_data['text'].apply(clean_html_and_text)
test_data['text'] = test_data['text'].apply(clean_html_and_text)

# Remove rows with empty text after preprocessing
train_data = train_data[train_data['text'].str.strip() != '']
test_data = test_data[test_data['text'].str.strip() != '']

In [49]:
# Display basic information about the datasets
print("Train Data Shape:", train_data.shape)
print("Test Data Shape:", test_data.shape)
print("Train Data Sample:")
print(train_data.head())
print("Test Data Sample:")
print(test_data.head())


Train Data Shape: (983, 2)
Test Data Shape: (987, 1)
Train Data Sample:
                                                text  label
0  dear sir strictly private business proposal mi...      1
2  noracheryl emailed dozen memo haiti weekend pl...      0
3  dear sirfmadamc know proposal might surprise e...      1
4                                                fyi      0
5  sure bottom line need special security code ge...      0
Test Data Sample:
                                                text
0  usiness fact deceased man foreigner authorized...
1  happy adjust afternoon going suggest pm start ...
2  lael brainard confirmed afternoonmiguel rodrig...
3      friday march amsbwhoeop rei extended congrats
4  dear good friendi happy inform successin getti...


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [50]:
# Check if categories contain any data
if ham_words.empty:
    print("No data available for 'ham' messages after preprocessing.")
else:
    count_vectorizer = CountVectorizer()
    ham_bow = count_vectorizer.fit_transform(ham_words)
    ham_word_counts = pd.DataFrame({
        'word': count_vectorizer.get_feature_names_out(),
        'count': ham_bow.sum(axis=0).A1
    }).sort_values(by='count', ascending=False).head(10)
    print("Top 10 words in ham messages:")
    print(ham_word_counts)

if spam_words.empty:
    print("No data available for 'spam' messages after preprocessing.")
else:
    count_vectorizer = CountVectorizer()
    spam_bow = count_vectorizer.fit_transform(spam_words)
    spam_word_counts = pd.DataFrame({
        'word': count_vectorizer.get_feature_names_out(),
        'count': spam_bow.sum(axis=0).A1
    }).sort_values(by='count', ascending=False).head(10)
    print("Top 10 words in spam messages:")
    print(spam_word_counts)

No data available for 'ham' messages after preprocessing.
No data available for 'spam' messages after preprocessing.


## Extra features

In [53]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€","$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

train_data['money_mark'] = train_data['text'].str.contains(money_simbol_list)*1
train_data['suspicious_words'] = train_data['text'].str.contains(suspicious_words)*1
train_data['text_len'] = train_data['text'].apply(lambda x: len(x)) 

test_data['money_mark'] = test_data['text'].str.contains(money_simbol_list)*1
test_data['suspicious_words'] = test_data['text'].str.contains(suspicious_words)*1
test_data['text_len'] = test_data['text'].apply(lambda x: len(x)) 

train_data.head()

,text,label,money_mark,suspicious_words,text_len
0,dear sir strictly private business proposal mi...,1,1,1,1498
2,noracheryl emailed dozen memo haiti weekend pl...,0,1,0,110
3,dear sirfmadamc know proposal might surprise e...,1,1,1,1378
4,fyi,0,1,0,3
5,sure bottom line need special security code ge...,0,1,0,311


## How would work the Bag of Words with Count Vectorizer concept?

In [55]:
# Exploratory Data Analysis: Get top 10 words in ham and spam messages
ham_words = train_data[train_data['label'] == 'ham']['text']
spam_words = train_data[train_data['label'] == 'spam']['text']

count_vectorizer = CountVectorizer()
ham_bow = count_vectorizer.fit_transform(ham_words)
spam_bow = count_vectorizer.fit_transform(spam_words)

ham_word_counts = pd.DataFrame({
    'word': count_vectorizer.get_feature_names_out(),
    'count': ham_bow.sum(axis=0).A1
}).sort_values(by='count', ascending=False).head(10)

spam_word_counts = pd.DataFrame({
    'word': count_vectorizer.get_feature_names_out(),
    'count': spam_bow.sum(axis=0).A1
}).sort_values(by='count', ascending=False).head(10)

print("Top 10 words in ham messages:")
print(ham_word_counts)

print("Top 10 words in spam messages:")
print(spam_word_counts)

ValueError: empty vocabulary; perhaps the documents only contain stop words

## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [56]:
# Convert text data into numerical features using TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

## And the Train a Classifier?

In [ ]:
# Your code

### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).

In [ ]:
# Your code